In [49]:
# File imports
!wget https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-4923/ud-treebanks-v2.11.tgz?sequence=1&isAllowed=y
!tar -xvf /content/ud-treebanks-v2.11.tgz?sequence=1
!cp /content/ud-treebanks-v2.11/UD_English-Atis/en_atis-ud-dev.conllu .
!cp /content/ud-treebanks-v2.11/UD_English-Atis/en_atis-ud-train.conllu .
!cp /content/ud-treebanks-v2.11/UD_English-Atis/en_atis-ud-test.conllu .

--2023-03-09 10:22:50--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-4923/ud-treebanks-v2.11.tgz?sequence=1
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 514078349 (490M) [application/x-gzip]
Saving to: ‘ud-treebanks-v2.11.tgz?sequence=1.2’

ud-treebanks-v2.11. 100%[===================>] 490.26M  17.3MB/s    in 32s     

2023-03-09 10:23:23 (15.5 MB/s) - ‘ud-treebanks-v2.11.tgz?sequence=1.2’ saved [514078349/514078349]

ud-treebanks-v2.11/
ud-treebanks-v2.11/UD_German-LIT/
ud-treebanks-v2.11/UD_German-LIT/stats.xml
ud-treebanks-v2.11/UD_German-LIT/README.md
ud-treebanks-v2.11/UD_German-LIT/de_lit-ud-test.conllu
ud-treebanks-v2.11/UD_German-LIT/LICENSE.txt
ud-treebanks-v2.11/UD_German-LIT/de_lit-ud-test.txt
ud-treebanks-v2.11/UD_Faroese-OFT/
ud-treebanks-v2.11/UD_Faroese-OFT/LICENSE.txt
ud-treebanks-

In [50]:
!pip3 install conllu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
import warnings
warnings.filterwarnings('ignore')


import nltk
import torchtext
import torch
import torch.nn as nn
import conllu as cn
import os
import sys
import numpy as np
## imports 

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_
import os

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', device)

Device available now: cpu


In [53]:
from torchtext import data
import torch.nn.functional as F
class TextProcessing():

  def __init__(self,path):

    self.path=path
    self.sentence_list=[]
    self.tag_list=[]
    self.id_list=[]
    self.sentence_l=None
    self.tag_l=None

  def pass_val(self,it):

    if(it==None):
      return
    self.sentence_list.append(it.token['form'])
    self.tag_list.append(it.token['upos'])
    self.id_list.append(it.token['id'])
    if(it.children==None):
      return
    for val in range(len(it.children)):
      self.pass_val(it.children[val])

  def read_file(self):

    data_read=open(self.path,"r",encoding="utf-8").read()
    sentence_l=[]
    tag_l=[]
    x=cn.parse_tree(data_read)
    for tokenlist in range(len(x)):
        self.pass_val(x[tokenlist])
        val=[x for _, x in sorted(zip(self.id_list, self.sentence_list))]
        sentence_l.append(val)
        val=[x for _, x in sorted(zip(self.id_list, self.tag_list))]
        tag_l.append(val)
        self.id_list=[]
        self.sentence_list=[]
        self.tag_list=[]
    self.sentence_l=sentence_l;self.tag_l=tag_l
    return sentence_l,tag_l

  def vocab_builder(self,given_l,pad_build=True):
    given_vocab=dict()
    if(pad_build==True):
       given_vocab["<pad>"]=1000
    for x in given_l:
      for y in x:
        if y in given_vocab:
          given_vocab[y]+=1
        else:
          given_vocab[y]=1
    
    return given_vocab

  def unk_builder(self,given_dict,min_freq=1):

    given_dict['unk']=0
    for key in list(given_dict.keys()):
      if(given_dict[key]<=min_freq):
        given_dict['unk']+=given_dict[key]
        del given_dict[key]
    return given_dict

def map_dict_to_vals(given_dict,given_list):

    final_l = []
    copy_dict={}
    for idx,val in enumerate(list(given_dict.keys())):
      copy_dict[val]=idx
    for sentence in given_list:
      hack_list=[]
      for word in sentence:
        if word in copy_dict:
          hack_list.append(copy_dict[word])
        else:
          hack_list.append(copy_dict['unk'])
      final_l.append(np.array(hack_list))
    # final_l=np.array(final_l)
    return final_l,copy_dict # returns numpy array not a tensor lol

def tensor_prep(given_tensor,PAD_VALUE=0):

    max_length = max(len(row) for row in given_tensor)
    result = np.array([np.pad(row, (PAD_VALUE, max_length-len(row))) for row in given_tensor])
    result=torch.from_numpy(result)
    return result

def preprocessing(path):
      ### train sentence tensor #####

      txt_process=TextProcessing(path)
      train_sentence,train_tag=txt_process.read_file()
      dict_sentence=txt_process.vocab_builder(train_sentence)
      unk_sentence=txt_process.unk_builder(dict_sentence)
      sentence_np,sentence_dict_indexes=map_dict_to_vals(unk_sentence,train_sentence)
      sentence_tensor=tensor_prep(sentence_np)

      #### tag sentence tensor ######

      dict_tag=txt_process.vocab_builder(train_tag,True)
      tag_np,tag_dict_indexes=map_dict_to_vals(dict_tag,train_tag)
      tag_tensor=tensor_prep(tag_np)

      return sentence_tensor,tag_tensor,unk_sentence,dict_tag,sentence_dict_indexes,tag_dict_indexes
# "/content/en_atis-ud-train.conllu"
sentence_train_tensor,tag_train_tensor,sentence_vocab,tag_vocab,sentence_dict_indexes,tag_dict_indexes=preprocessing("/content/en_atis-ud-train.conllu")

Model

In [54]:
class BiLSTM(nn.Module):

  def __init__(self, in_dim, em_dim, hid_dim, out_dim, n_layers, dropout, pad_idx = 0):
    
    super(BiLSTM, self).__init__()

    self.embedding = nn.Embedding(in_dim, em_dim, padding_idx = pad_idx)
    self.lstm = nn.LSTM(em_dim, hid_dim, num_layers=n_layers, batch_first=True, bidirectional=True)
    self.fc = nn.Linear(hid_dim*2, out_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):

    x = self.embedding(x)
    out, vals = self.lstm(x)
    out = self.fc(self.dropout(out))

    return out

Train Model

In [55]:
in_dim=len(sentence_vocab)
mid_dim=100
lstm_dim=128
out_dim=len(tag_vocab)
n_layers=4
dropout_num=0.15
PAD_IDX=0
model=BiLSTM(in_dim,mid_dim,lstm_dim,out_dim,n_layers,dropout_num).to(device)



In [56]:
optimizer = optim.Adam(model.parameters(),lr=0.01,weight_decay=1e-4)
loss = nn.CrossEntropyLoss(ignore_index = 0).to(device)

In [57]:

BATCH_SIZE=20

# def epoch_time(start_time, end_time):
#     elapsed_time = end_time - start_time
#     elapsed_mins = int(elapsed_time / 60)
#     elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
#     return elapsed_mins, elapsed_secs




In [58]:
N_EPOCHS = 10
# import time
# best_valid_loss = float('inf')

# for epoch in range(N_EPOCHS):

#     start_time = time.time()
    
    # train_loss, train_acc = train(model, sentence_train_tensor, optimizer, loss, 0,tag_train_tensor)
    # valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)
    
    # end_time = time.time()

    # epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(model.state_dict(), 'tut1-model.pt')
    
    # print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    # print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    # print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')



In [59]:
# def train(model,criterion,optimizer,input_data_sen, input_data_tag,batch_size,num_epochs, tag_pad_idx=0):
#   # min_loss=1000000.0
#   for epoch in range(num_epochs):
#     model.zero_grad()
#     epoch_loss = 0
#     epoch_acc = 0
#     # states = (torch.zeros(num_layers*2, input_data_sen.size(0), hidden_dim).to(device),
#     #           torch.zeros(num_layers*2, input_data_sen.size(0), hidden_dim).to(device))
#     for ind, i in enumerate(input_data_sen):
#       inputs = input_data_sen[ind].to(device)
#       targets = input_data_tag[ind].to(device)

#       # forward pass
#       # states = detach(states)
#       outputs = model(inputs)

#       outputs = outputs.view(-1, outputs.shape[-1])
#       targets = targets.view(-1)

#       loss = criterion(outputs, targets)

#       # backward pass
#       optimizer.zero_grad()
#       loss.backward()
#       optimizer.step()

#       epoch_loss += loss.item()

#     epoch_loss = epoch_loss / len(input_data_sen)

#     # if(epoch_loss<min_loss):
#     #     min_loss=epoch_loss
#     #     torch.save(model.state_dict(), 'model_pos_tagger.pth')


#     print(f'Epoch {epoch}/{num_epochs}\t Loss {epoch_loss}')

# train(model, loss, optimizer, sentence_train_tensor, tag_train_tensor, BATCH_SIZE, N_EPOCHS)


In [60]:
from sklearn.metrics import precision_recall_fscore_support,f1_score,accuracy_score
  # def init_weights(m):
  #     for name, param in m.named_parameters():
  #         nn.init.normal_(param.data, mean = 0, std = 0.1)
        
# model.apply(init_weights)

# class ModelHelper():

#   def __init__(self,model,loss,optim,batch_size,epochs,tag_pad_idx=0):

#     self.model=model
#     self.loss=loss
#     self.optim=optim
#     self.batch_size=batch_size
#     self.epochs=epochs
#     self.tag_pad_idx=tag_pad_idx

#   def train(self,input_data_sen, input_data_tag):
  
#     for epoch in range(self.epochs):
#       self.model.zero_grad()
#       epoch_loss = 0
#       epoch_acc = 0

#       for ind, i in enumerate(input_data_sen):
#         inputs = input_data_sen[ind].to(device)
#         targets = input_data_tag[ind].to(device)

#         outputs = self.model(inputs)

#         outputs = outputs.view(-1, outputs.shape[-1])
#         targets = targets.view(-1)

#         loss = self.loss(outputs, targets)

#         # backward pass
#         self.optim.zero_grad()
#         self.loss.backward()
#         self.optim.step()

#         epoch_loss += self.loss.item()

#       epoch_loss = epoch_loss / len(input_data_sen)
#       print(f'Epoch {epoch}/{self.num_epochs}\t Loss {epoch_loss}')

#   def save_model(self):
#     torch.save(self.model.state_dict(), 'model_pos_tagger.pth')

class Evaluate():
  def __init__(self,model,average='weighted'):
    self.model=model
    self.average=average
  def categorical_accuracy(self,preds, y,tag_pad_idx=0):
    
          """
          Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
          """
          max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
          non_pad_elements = (y != tag_pad_idx).nonzero()
          pred=max_preds[non_pad_elements].squeeze(1)
          actual=y[non_pad_elements]
          return self.return_scores(actual,pred)

  def return_scores(self,actual,pred):
          # correct = pred.eq(actual)
          acc_score=accuracy_score(actual,pred)
          score=f1_score(actual,pred,average=self.average, labels=np.unique(pred))
          prec_recall=precision_recall_fscore_support(actual,pred,average=self.average, labels=np.unique(pred))
          return acc_score,score,prec_recall[0],prec_recall[1]

  def evaluate(self, input_data_sen, input_data_tag, criterion, tag_pad_idx=0):
        
        self.model.eval()
        epoch_loss = 0
        epoch_acc = 0
        epoch_f1=0
        epoch_prec=0
        epoch_recall=0
        
        with torch.no_grad():
        
            for ind, batch in enumerate(input_data_sen):

              targets = input_data_tag[ind].to(device)
              targets = targets.reshape(-1)

              inputs = batch.to(device)
              predictions = self.model(inputs)
              shape_of_tensor=predictions.shape
              predictions = predictions.reshape(-1,shape_of_tensor[-1])

              loss = criterion(predictions, targets)
              acc,f1,precision,recall = self.categorical_accuracy(predictions, targets)
              epoch_loss += loss.item()
              epoch_acc += acc.item()
              epoch_f1+=f1.item()
              epoch_prec+=precision.item()
              epoch_recall+=recall.item()

        return  epoch_acc/len(input_data_sen),epoch_f1/len(input_data_sen),epoch_prec/len(input_data_sen),epoch_recall/len(input_data_sen)

# Load Model

In [61]:
from google.colab import drive
drive.mount('/content/drive')
# !cp  .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
model.load_state_dict(torch.load("/content/drive/MyDrive/model_pos_tagger.pth",map_location=torch.device('cpu')))


<All keys matched successfully>

In [63]:
# sentence_test_tensor,tag_test_tensor,_,_=preprocessing("/content/en_atis-ud-test.conllu")


In [64]:
def test_prep(path):
  txt_process=TextProcessing(path)
  test_sentence,test_tag=txt_process.read_file()
  sentence_np,_=map_dict_to_vals(sentence_vocab,test_sentence)
  sentence_tensor=tensor_prep(sentence_np)

  tag_np,_=map_dict_to_vals(tag_vocab,test_tag)
  tag_tensor=tensor_prep(tag_np)

  return sentence_tensor,tag_tensor

sentence_test_tensor,tag_test_tensor=test_prep("/content/en_atis-ud-test.conllu")
ev=Evaluate(model,average='macro')
ev.evaluate( sentence_test_tensor, tag_test_tensor, loss)


(0.9336723815329112,
 0.9330802475371537,
 0.9339073677442152,
 0.9444480488699769)

# Inference of Model

In [65]:
import regex as re

inv_tag_map = {v: k for k, v in tag_dict_indexes.items()}
inv_sentence_map = {v: k for k, v in sentence_dict_indexes.items()}

# in given list pass only one sentence
def reverse_dict_map(given_dict,given_list):
  result=[]
  for key in given_list:
    result.append(given_dict[key])
  return result

def infer_sentence(sent,path=""):

    sent_arr=[]
    sent_arr.append(re.split(" ",sent.lower()))
    txt_process_=TextProcessing(path)
    sentence_np,_=map_dict_to_vals(sentence_vocab,sent_arr)
    sentence_tensor=tensor_prep(sentence_np)
    inputs=sentence_tensor.to(device)
    predictions = model(inputs)
    top_predictions = predictions.argmax(-1)[0].cpu().detach().numpy()
    predicted_tags=reverse_dict_map(inv_tag_map,top_predictions)
    
    return predicted_tags
  
og_arr=tag_train_tensor[0].cpu().detach().numpy() 
tag_fixed=np.delete(og_arr, np.where(og_arr == 0))
# sentence="what is the cost of a round trip flight from pittsburgh to atlanta beginning on april twenty fifth and returning on may sixth"
# The above given sample is from train set
sentence=input("Enter Sentence for to get output")
sentence = " ".join(re.findall("[a-zA-Z]+", sentence))

predicted_tags=infer_sentence(sentence)
arr=re.split(" ",sentence)
for idx,word in enumerate(arr):
  print(word+"\t"+predicted_tags[idx])


Enter Sentence for to get outputwhat is the cost of a round trip flight from pittsburgh to atlanta beginning on april twenty fifth and returning on may sixth
what	PRON
is	AUX
the	DET
cost	NOUN
of	ADP
a	DET
round	NOUN
trip	NOUN
flight	NOUN
from	ADP
pittsburgh	PROPN
to	ADP
atlanta	PROPN
beginning	PROPN
on	ADP
april	NOUN
twenty	INTJ
fifth	ADJ
and	CCONJ
returning	VERB
on	ADP
may	PROPN
sixth	PROPN
